## Day 49 Lecture 1 Assignment

In this assignment, we will apply GMM (Gaussian Mixture Modeling) clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.special import entr

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [3]:
# answer goes here
df = pd.read_csv("https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv")
df.head()

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,0.357,1.3,4.1,0.323,0.5,1.0,0.500,0.487,0.4,0.4,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2019,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,0.222,0.2,1.5,0.133,0.2,0.3,0.667,0.278,0.7,1.0,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,2019,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,0.345,0.7,2.2,0.338,0.4,1.1,0.361,0.459,0.2,0.3,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,0.596,0.595,1.8,3.7,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,0.588,0.579,2.0,2.8,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9


The goal is to cluster these player-seasons to identify potential player "archetypes".  
The pre-processing steps will be identical to what we previously did for K-means.

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [74]:
# answer goes here
nba = df.loc[(df.MP >= 10) & (df.GS >= 20)]
nba.shape

(804, 30)

Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [35]:
# answer goes here
X = nba[["PTS", "TRB", "AST", "STL", "BLK", "TOV"]]

Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [36]:
# answer goes here
scaler = StandardScaler()
X_scale = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

Run both K-Means and Gaussian mixtures modeling twice, once with 3 cluster and once with 7 clusters. Print out the resulting means for all 4 scenarios (KM+3, GMM+3, KM+7, GMM+7). When printing the means, transform the scaled versions back into their corresponding unscaled values. 

What "archetypes" do you see? Are the archetypes identified by GMM similar to those identified by K-Means? How do the means of GMM differ from those of K-Means?

In [46]:
print("KM+3")
X = nba[["PTS", "TRB", "AST", "STL", "BLK", "TOV"]]
X_scale = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

kmeans3 = KMeans(n_clusters=3)
X_scale["cluster"] = kmeans3.fit_predict(X_scale)

centers = pd.DataFrame(kmeans3.cluster_centers_, columns=X.columns)
inverse = pd.DataFrame(scaler.inverse_transform(centers), columns=X.columns)
inverse.style.background_gradient()

KM+3


,PTS,TRB,AST,STL,BLK,TOV
0,10.505405,3.949775,2.063739,0.804730,0.374324,1.235135
1,13.735359,8.605525,1.914917,0.777348,1.230387,1.668508
2,19.491620,5.293855,5.934078,1.354190,0.486034,2.797765


In [49]:
print("KM+7")
X = nba[["PTS", "TRB", "AST", "STL", "BLK", "TOV"]]
X_scale = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

kmeans7 = KMeans(n_clusters=7)
X_scale["cluster"] = kmeans7.fit_predict(X_scale)

centers = pd.DataFrame(kmeans7.cluster_centers_, columns=X.columns)
inverse = pd.DataFrame(scaler.inverse_transform(centers), columns=X.columns)
inverse.style.background_gradient()

KM+7


,PTS,TRB,AST,STL,BLK,TOV
0,11.785714,7.429252,1.734014,0.663265,0.937415,1.431293
1,7.593048,3.544920,1.360963,0.594118,0.375401,0.889840
2,24.603226,8.374194,7.803226,1.603226,0.809677,3.941935
3,18.457600,4.676800,5.788000,1.343200,0.401600,2.624000
4,14.304819,3.709639,3.169880,0.837952,0.277108,1.715060
5,17.765517,10.512069,2.291379,0.925862,1.770690,2.141379
6,11.246667,4.824444,2.124444,1.348889,0.487778,1.281111


In [50]:
print("GMM+3")
X = nba[["PTS", "TRB", "AST", "STL", "BLK", "TOV"]]
X_scale = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

gmm_cluster3 = GaussianMixture(n_components=3)
X_scale["cluster"] = gmm_cluster3.fit_predict(X_scale)

centers = pd.DataFrame(gmm_cluster3.means_, columns=X.columns)
inverse = pd.DataFrame(scaler.inverse_transform(centers), columns=X.columns)
inverse.style.background_gradient()

GMM+3


,PTS,TRB,AST,STL,BLK,TOV
0,14.550186,8.524727,2.279152,0.777242,1.230885,1.842771
1,10.228179,4.504354,1.579742,0.779853,0.433706,1.107959
2,15.895675,4.173718,4.807518,1.176113,0.370977,2.244243


In [65]:
print("GMM+7")
X = nba[["PTS", "TRB", "AST", "STL", "BLK", "TOV"]]
X_scale = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

gmm_cluster7 = GaussianMixture(n_components=7)
X_scale["cluster"] = gmm_cluster7.fit_predict(X_scale)

centers = pd.DataFrame(gmm_cluster7.means_, columns=X.columns)
inverse = pd.DataFrame(scaler.inverse_transform(centers), columns=X.columns)
inverse.style.background_gradient()

GMM+7


,PTS,TRB,AST,STL,BLK,TOV
0,14.871119,3.238720,4.359562,0.929819,0.219538,2.027738
1,14.042871,6.149907,1.829701,0.702119,0.449498,1.375634
2,8.517667,2.945636,1.496592,0.715729,0.269600,0.887238
3,17.326843,9.513854,2.932958,0.876759,1.419379,2.195483
4,17.718459,5.584173,5.996867,1.550551,0.548079,2.715573
5,8.904228,5.737615,1.167249,0.642320,0.912127,1.106135
6,11.677400,4.957025,2.205605,1.131331,0.470059,1.476168


*Archetyping between the two models are different. In +7, the best offensive players in GMM comes at cluster 4 while in KM they are at 6. It is also notable that total rebounds and blocks are not the highest on those indices, indicating that those on cluster 0 on GMM and cluster 2 on KM are the best defensive players. On +3, KM has the best offensive players on 0 and GMM has it on 2.*

Predict the likelihood of each player belonging to one of the 3 clusters using the GMM model. Then, calculate the entropy for each set of predicted probabilities. 

We will use entropy as a measure of how confident we are in the predicted class label. If we had no confidence in our prediction, we would assign 33% probability to each class, while if we were totally confident, we would assign 100% to one class. Entropy would be at a maximum in the "no confidence" scenario and a minimum in the "full confidence" scenario, which makes it a reasonable way to quantify our uncertainty in our prediction. There are certainly other methods as well; feel free to experiment with them if desired.

Which five predicted labels are we least confident about? Which five are we most confident about? Print out the associated details (season, player name, stats, etc.) from those players.

In [72]:
# answer goes here
X = nba[["PTS", "TRB", "AST", "STL", "BLK", "TOV"]]
X_scale = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
probs = pd.DataFrame(gmm_cluster3.predict_proba(X_scale))
ent = entr(probs)

In [93]:
tot = pd.DataFrame(ent.sum(axis=1), columns=['Total Entropy'])
top5 = tot.sort_values(by='Total Entropy', ascending=False).head().index

In [92]:
least5 = tot.sort_values(by='Total Entropy', ascending=True).head().index

In [127]:
print("5 with the Most Total Entropies (0% Certainty)")
ref = pd.DataFrame(columns=["Season", "Player", "Position", "Age", "Team"])

for i in top5:
    playerdata = []

    playerdata.append(nba.iloc[i].Seas)
    playerdata.append(nba.iloc[i].Player)
    playerdata.append(nba.iloc[i].Pos)
    playerdata.append(nba.iloc[i].Age)
    playerdata.append(nba.iloc[i].Tm)
    ref.loc[i] = playerdata

ref

5 with the Most Total Entropies


,Season,Player,Position,Age,Team
13,2019,Will Barton,SG,28,DEN
741,2016,Markieff Morris,PF,26,TOT
343,2018,Kelly Olynyk,PF,26,MIA
109,2019,James Johnson,PF,31,MIA
406,2017,Carmelo Anthony,SF,32,NYK


In [128]:
print("5 with the Least Total Entropies (100% Certainty)")
ref = pd.DataFrame(columns=["Season", "Player", "Position", "Age", "Team"])

for i in least5:
    playerdata = []

    playerdata.append(nba.iloc[i].Seas)
    playerdata.append(nba.iloc[i].Player)
    playerdata.append(nba.iloc[i].Pos)
    playerdata.append(nba.iloc[i].Age)
    playerdata.append(nba.iloc[i].Tm)
    ref.loc[i] = playerdata

ref

5 with the Least Total Entropies (100% Certainty)


,Season,Player,Position,Age,Team
797,2016,Hassan Whiteside,C,26,MIA
182,2019,Myles Turner,C,22,IND
252,2018,Anthony Davis,PF,24,NOP
351,2018,Kristaps Porziņģis,PF,22,NYK
454,2017,Joel Embiid,C,22,PHI
